In [1]:
data_var = '2023-12-12'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7956,2023-12-12,Finlândia Korisliiga,13:30,KTP Basket,Kouvot,1.54,2.41,179.5,1.80,1.86,-5.5,2.09,1.66,zcab6sU9,0.649351,0.414938,0.555556,0.537634,0.064288,144.134,36.038395,0.250034,1.2,1.643168,1.369306,193.27,1.682,0.514121,0.305660,6.0,190.402,73.302184,0.384986,1.8,1.643168,0.912871,131.97,2.190,0.931692,0.425430,-3.0,77,83,2.51,1.59,0.000,0.000,0.311485,0.023184,0.162163,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7957,2023-12-12,Europa Euroliga Feminina,16:00,Lyon F,Fenerbahce F,8.65,1.06,156.5,1.81,1.85,18.5,2.02,1.71,QN7lJPQm,0.115607,0.943396,0.552486,0.540541,0.059003,220.688,220.506838,0.999179,0.6,1.341641,2.236068,176.64,3.542,3.956680,1.117075,-35.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,64,88,2.76,1.06,0.000,0.000,1.105446,0.015456,0.117535,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7958,2023-12-12,Europa Liga Letônia,14:00,Tartu Ulikool,Keila,1.26,3.61,158.5,1.85,1.85,-9.5,2.00,1.72,ze6GmFR4,0.793651,0.277008,0.540541,0.540541,0.070659,234.510,287.310571,1.225153,1.2,1.643168,1.369306,86.62,3.198,4.085403,1.277487,-8.0,328.292,202.919027,0.618105,1.2,1.643168,1.369306,440.20,4.514,3.703239,0.820390,-53.0,71,71,1.22,6.20,0.000,0.000,0.682423,0.000000,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7959,2023-12-12,Europa Liga Letônia,14:00,BK Liepaja,Rigas Zelli,2.59,1.46,154.5,1.85,1.85,3.5,2.02,1.70,lbAKnZtB,0.386100,0.684932,0.540541,0.540541,0.071032,163.410,106.019308,0.648793,1.2,1.643168,1.369306,123.00,2.138,1.352043,0.632387,0.0,134.708,80.575054,0.598146,2.4,1.341641,0.559017,112.14,1.762,1.407576,0.798851,55.0,75,89,1.64,1.26,0.000,0.000,0.394583,0.000000,0.121653,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
7960,2023-12-12,Europa Enbl,14:00,Spartak Pleven,Juventus,6.36,1.09,160.5,1.85,1.85,12.5,2.00,1.72,SW5DojG1,0.157233,0.917431,0.540541,0.540541,0.074664,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,71,0,3.30,0.00,0.000,0.000,1.000390,0.000000,0.106446,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8019,2023-12-12,Eua Ncaa,21:00,Duke,Hofstra,1.07,9.80,143.5,1.80,1.95,-16.5,2.10,1.67,0pSH7Xb4,0.934579,0.102041,0.555556,0.512821,0.036620,109.266,30.786073,0.281753,1.8,1.643168,0.912871,84.80,1.384,0.342462,0.247443,63.0,124.146,19.316179,0.155592,2.4,1.341641,0.559017,105.40,1.584,0.122393,0.077268,35.0,80,68,1.06,1.55,0.000,123.958,1.135794,0.056569,0.161303,0.00,0.000,inf,0.0,0.0,0.0,-0.40,-0.080,-110.000000,0.0,0.0,0.0
8020,2023-12-12,Eua Ncaa,21:00,Cincinnati,Bryant University,1.04,13.50,151.5,1.80,1.95,-20.5,2.10,1.67,EyfBTul2,0.961538,0.074074,0.555556,0.512821,0.035613,109.790,18.205969,0.165825,2.4,1.341641,0.559017,142.20,1.256,0.308432,0.245566,96.0,271.598,331.652337,1.221115,1.8,1.643168,0.912871,150.00,3.718,4.366780,1.174497,-11.0,79,75,1.80,2.00,98.416,444.956,1.211905,0.056569,0.161303,0.40,0.080,0.500000,0.0,0.0,0.0,14.46,2.892,

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V2 (LR)

In [9]:
# Duplicando o dataset
df_bh_v2 = jogos.copy()

df_bh_v2 = df_bh_v2[(df_bh_v2['Odds_H'] >= 1.6) & (df_bh_v2['Odds_H'] <= 2.4)]
df_bh_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v2.empty:
    mdl_bh_v2 = load_model('ML/back_home/v2/v2_back_home', verbose=False)
    prev = predict_model(mdl_bh_v2, df_bh_v2)
    prev = prev[prev['prediction_label'] == 1]

    # Filtros pós modelo
    prev = prev[((prev['CV_Odds_Geral_H'] <= 0.224) | (prev['CV_Odds_Geral_H'] > 0.441))]
    prev = prev[((prev['P(U)'] < 0.526) | (prev['P(U)'] > 0.535))]
    prev = prev[((prev['Last_Odd_H'] <= 1.59) | (prev['Last_Odd_H'] > 2.3))]

    if not df_bh_v2.empty:
        df_bh_v2.to_csv(f'apostas_do_dia/{data_var}/back_home_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V2 (LR) ------------')
        print()

        df_bh_v2['Bet'] = 'Back Home'
        ligs = df_bh_v2['League'].unique().tolist()

        display(df_bh_v2[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO BACK HOME V2 (LR) ------------



,Time,League,Home,Away,Odds_H,Bet
7962,16:30,Europa Liga Dos Campeões,Oostende,Strasbourg,2.10,Back Home
7964,14:30,Europa Liga Dos Campeões,Promitheas,Dijon,1.67,Back Home
7971,14:00,Lituânia Nkl,Vilniaus Stekas,Vilkaviskis,1.94,Back Home
7974,15:04,Suécia Basketligan,Uppsala,BC Lulea,2.31,Back Home
7975,15:04,Suécia Basketligan,Umeå IK,Koping Stars,2.05,Back Home
7982,13:00,Lituânia Nkl,Kauno Atletas,Zalgiris Kaunas 2,1.94,Back Home
7983,15:45,Israel Liga Leumit,Elitzur Shomron,Ironi Nahariya,1.89,Back Home
7985,14:00,Israel Liga Leumit,Elitzur Ashkelon,Elitzor Netania,1.66,Back Home
7986,17:15,Islândia Premier League Feminina,Grindavik F,Njardvik F,2.07,Back Home
7995,21:30,Eua Nba,Dallas Mavericks,Los Angeles Lakers,2.11,Back Home


### Under V1 (QCut)

In [10]:
# Duplicando o dataset
df_u_v1 = jogos.copy()

df_u_v1 = df_u_v1[(df_u_v1['Odds_H'] >= 1.6) & (df_u_v1['Odds_H'] <= 2.4)]

if not df_u_v1.empty:    
    # Filtros pós modelo
    df_u_v1 = df_u_v1[((df_u_v1['CG_A_Last_Game'] <= 191) | (df_u_v1['CG_A_Last_Game'] > 280))]
    df_u_v1 = df_u_v1[((df_u_v1['Saldo_Gols_H'] <= 0) | (df_u_v1['Saldo_Gols_H'] > 16))]
    df_u_v1 = df_u_v1[((df_u_v1['CG_H_Last_Game'] <= 166.6) | (df_u_v1['CG_H_Last_Game'] > 260.85))]
    df_u_v1 = df_u_v1[((df_u_v1['Avg_CG_H'] <= 262.034) | (df_u_v1['Avg_CG_H'] > 969.8))]
    df_u_v1 = df_u_v1[((df_u_v1['MediaCustoGolAway'] <= 200.907) | (df_u_v1['MediaCustoGolAway'] > 868.35))]
    df_u_v1 = df_u_v1[((df_u_v1['P_Diff'] <= 0.0747) | (df_u_v1['P_Diff'] > 0.0971))]
    df_u_v1 = df_u_v1[((df_u_v1['HA_Odds_A'] >= 0) & (df_u_v1['HA_Odds_A'] <= 1.68)) | ((df_u_v1['HA_Odds_A'] > 1.8) & (df_u_v1['HA_Odds_A'] <= 1.9))]
    df_u_v1 = df_u_v1[((df_u_v1['CV_HA'] >= 0) & (df_u_v1['CV_HA'] <= 0.162))]
    df_u_v1 = df_u_v1[((df_u_v1['Last_Odd_A'] <= 1.297) | (df_u_v1['Last_Odd_A'] > 1.62))]
    df_u_v1 = df_u_v1[((df_u_v1['P(A)'] <= 0.234) | (df_u_v1['P(A)'] > 0.365))]
    
    if not df_u_v1.empty:
        df_u_v1.to_csv(f'apostas_do_dia/{data_var}/under_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO UNDER V1 (QCUT) ------------')
        print()

        df_u_v1['Bet'] = 'Back Home'
        ligs = df_u_v1['League'].unique().tolist()

        display(df_u_v1[['Time', 'League', 'Home', 'Away', 'Odds_Under', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos


### Back Home V3 (QCut)

In [11]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7974,15:04,Suécia Basketligan,Uppsala,BC Lulea,2.31,Back Home
7999,15:00,Europa Eurocopa,Ulm,Gran Canaria,2.19,Back Home
8003,13:00,Catar Liga De Basquete Do Catar,Catar,Al Gharafa,1.57,Back Home
